# Lab Exercise: Implement batch normalization and dropout

In this notebook you will modify your implementation of a convolutional neural network to include batch normalization and dropout of the dense layer. Then, you will validate the changes in your local environment, and finally train and deploy the model using Cloud ML Engine. The objective of this lab is to significantly improve the accuracy of your model.

In [ ]:
import os
PROJECT = 'my-project-id' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'my-bucket-name' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE='cnn_batch_norm'  #  'cnn_batch_norm' or 'dnn' or 'cnn'

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_TYPE'] = MODEL_TYPE
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

In [ ]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

**TODO: Add batch normalization to the last dense layer of your CNN**

Recall that to implement batch normalization using <code>tf.layers</code> API you need to ensure that the activation function is used after batch normalization is applied to your dense layer. Here's an example of batch normalization of the last dense layer (h3) of the convolutional neural network used in this lab:

<pre>
  h3 = tf.layers.dense(p2flat, 300, activation=None) #Activation to be added separately after batch normalization

  h3 = tf.layers.batch_normalization(h3, 
             training=(mode == tf.estimator.ModeKeys.TRAIN))
             
  h3_batch_normed = tf.nn.relu(h3)
</pre>

Also, note that <code>mode == tf.estimator.ModeKeys.TRAIN</code> evaluates to <code>True</code> only during model training, so that batch norming is bypassed when the model accuracy is evaluated or used for predictions.

Open <a href="fashionmodel/trainer">fashionmodel/trainer</a> to find the `model.py` file. Add batch normalization as part of your <code>cnn_batch_norm_model</code> method implementation.

**TODO: Add dropout to the batch normed layer of your CNN**

Dropout is another feature of the <code>tf.layers</code> library. As with batch normalization, dropout will only be applied during model training. Dropout can be used on a batch normed layer as in the code snippet below:

<pre>
  h3_with_dropout = tf.layers.dropout(h3_batch_normed,rate=0.1, 
                                         training=(mode == tf.estimator.ModeKeys.TRAIN))
                    
  ylogits = tf.layers.dense(h3_with_dropout, NCLASSES, activation=None)

</pre>

Open <a href="fashionmodel/trainer">fashionmodel/trainer</a> to find the `model.py` file. Add dropout to the batched normed layer your implemented in the previous cell in the `cnn_batch_norm_model` method.


## Run as a Python module

You should expect that training will take a bit longer than with the previous lab.

In [ ]:
%bash
rm -rf fashionmodel.tar.gz fashion_trained
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/fashionmodel/trainer \
   -- \
   --output_dir=${PWD}/fashion_trained \
   --train_steps=1 \
   --learning_rate=0.01 \
   --model=$MODEL_TYPE

**Make sure that local training completed successfully before training using Cloud ML Engine**

Next, go ahead and try training with Cloud ML Engine. Compare the accuracy of the model that uses batch norming and dropout to the accuracy of convolutional neural network you  trained in the previous lab.

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/fashion/trained_${MODEL_TYPE}
JOBNAME=fashion_${MODEL_TYPE}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/fashionmodel/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC_GPU \
   --runtime-version=$TFVERSION \
   -- \
   --output_dir=$OUTDIR \
   --train_steps=10000 --learning_rate=0.01 --train_batch_size=512 \
   --model=$MODEL_TYPE

## Monitoring training with TensorBoard

Use this cell to launch tensorboard

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/fashion/trained_{}'.format(BUCKET, MODEL_TYPE))

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

## Deploying and predicting with model

**Make sure that the previous traiining step is finished and deploy the model:**

In [ ]:
%bash
MODEL_NAME="fashion"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/fashion/trained_${MODEL_TYPE}/export/exporter | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

The previous step of deploying the model can take a few minutes. If it is successful, you should see an output similar to this one:

<pre>
Created ml engine model [projects/qwiklabs-gcp-27eb45524d98e9a5/models/fashion].
Creating version (this might take a few minutes)......
...................................................................................................................done.
</pre>

Next, download a local copy of the Fashion MNIST dataset to use with Cloud ML Engine for predictions.

In [ ]:
import tensorflow as tf
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
LABELS = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

To predict with the model, save one of the test images as a JavaScript Object Notation (JSON) file. Also, take a look at it as a graphic and notice the expected class value in the title.

In [ ]:
HEIGHT=28
WIDTH=28

IMGNO=5500 #CHANGE THIS to get different images

#Convert raw image data to a test.json file and persist it to disk
import json, codecs
jsondata = {'image': test_images[IMGNO].reshape(HEIGHT, WIDTH).tolist()}
json.dump(jsondata, codecs.open('test.json', 'w', encoding='utf-8'))

#Take a look at a sample image and the correct label from the test dataset
import matplotlib.pyplot as plt
plt.imshow(test_images[IMGNO].reshape(HEIGHT, WIDTH))
title = plt.title('{} / Class #{}'.format(LABELS[test_labels[IMGNO]], test_labels[IMGNO]))

Send the file to the prediction service and check whether the model you trained returns the correct prediction.

In [ ]:
%bash
gcloud ml-engine predict \
   --model=fashion \
   --version=${MODEL_TYPE} \
   --json-instances=./test.json

<pre>
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>